## 绑定机制

使用 bind_providers 机制实现动运行时的变量传递。

每个 Runnable 实例都有一个 provider_dict 和 一个 consumer_dict，用于导出和导入绑定的变量。

如果 a 和 b 都是 Runnable 实例，并且执行了绑定操作 `a.bind_providers(b)`，就意味着将 a.consumer_dict 与 b.provider_dict 建立动态绑定关系。

这会导致：

1. 在 a 内部使用 a.consumer_dict 计算时会动态提取 b.provider_dict 的变量值
2. 默认情况下，会自动将 b.provider_dict 也同时作为 a.exported 的一部份传递，除非被同名导出变量覆盖

默认情况下，Runnable 实例不导入任何变量，但会导出 last_input 和 last_output 。

如果你需要增加导出变量，可以直接在 _provider_dict 字典中增加。

### 绑定实例

In [13]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        self._last_input = prompt
        self._last_output = f'你说了 {prompt}'

a = A()
a.call("hi")
print("a.consumer_dict", a.consumer_dict)
print("a.provider_dict", a.provider_dict)

a.consumer_dict {}
a.provider_dict {'last_input': 'hi', 'last_output': '你说了 hi'}


**默认传递 consumer_dict 为 provider_dict**

In [14]:
b = A()
b.bind_providers(a)
print("b's binding:", b.providers)
print("b.consumer_dict", b.consumer_dict)
print("b.provider_dict", b.provider_dict)

b's binding: [(<A.4747965584>, {})]
b.consumer_dict {'last_input': 'hi', 'last_output': '你说了 hi'}
b.provider_dict {'last_input': 'hi', 'last_output': '你说了 hi'}


**新增一个 provider_dict 值**

In [15]:
b._provider_dict['image'] = "girl.png"
print("b.consumer_dict", b.consumer_dict)
print("b.provider_dict", b.provider_dict)

b.consumer_dict {'last_input': 'hi', 'last_output': '你说了 hi'}
b.provider_dict {'last_input': 'hi', 'last_output': '你说了 hi', 'image': 'girl.png'}


**如果 consumer 修改了自己的 provider_dict 则用新值替代 provider 的输出值**

In [16]:
b._last_output = "我什么也没说"
print("b.consumer_dict", b.consumer_dict)
print("b.provider_dict", b.provider_dict)

b.consumer_dict {'last_input': 'hi', 'last_output': '你说了 hi'}
b.provider_dict {'last_input': 'hi', 'last_output': '我什么也没说', 'image': 'girl.png'}


**但如果设置为 None 则会重新采用 provider 的输出值**

In [17]:
b._last_output = None
print("b.consumer_dict", b.consumer_dict)
print("b.provider_dict", b.provider_dict)

b.consumer_dict {'last_input': 'hi', 'last_output': '你说了 hi'}
b.provider_dict {'last_input': 'hi', 'last_output': '你说了 hi', 'image': 'girl.png'}


**实际上，上游 provider 中设置为 None 的值不会被传递**

In [18]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        self._last_input = prompt
        self._last_output = None

a = A(name="A")
a.call("hi")
print("a.consumer_dict", a.consumer_dict)
print("a.provider_dict", a.provider_dict)

a.consumer_dict {}
a.provider_dict {'last_input': 'hi'}


## 使用映射规则

**提供了映射规则之后，将按照新的键名传递变量**

### 一般绑定

In [19]:
c = A(name="C")
c.bind_providers((a, {"task": "last_input"}))
print("a.provider_dict", a.provider_dict)
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

a.provider_dict {'last_input': 'hi'}
c.consumer_dict {'task': 'hi'}
c.provider_dict {'task': 'hi'}


**也可以使用函数来扩展映射时的逻辑**

使用函数扩展时，不会覆盖函数中包含的键值，这实际上提供了 **1:N** 映射的可能性。

In [21]:
c = A()
c.bind_providers((a, {"task": lambda x: "我来自 @A " + x["last_input"]}))
print("a.provider_dict", a.provider_dict)
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

a.provider_dict {'last_input': 'hi'}
c.consumer_dict {'last_input': 'hi', 'task': '我来自 @A hi'}
c.provider_dict {'last_input': 'hi', 'task': '我来自 @A hi'}


In [22]:
c.providers

[(<A>, {'task': <function __main__.<lambda>(x)>})]

### 避免重复绑定

In [23]:
c.bind_providers((a, {"task": "last_input"}))
print(a.consumers)
print(c.consumers)
print(a.providers)
print(c.providers)

[(<A C>, {'task': 'last_input'}), (<A.4747960448>, {'task': <function <lambda> at 0x11b380e50>}), (<A.4747966352>, {'task': <function <lambda> at 0x11b3823b0>})]
[]
[]
[(<A>, {'task': <function <lambda> at 0x11b3823b0>})]


In [24]:
a.bind_consumers(c, binding_map={"task": "last_input"})
print(a.consumers)
print(c.consumers)
print(a.providers)
print(c.providers)

[(<A C>, {'task': 'last_input'}), (<A.4747960448>, {'task': <function <lambda> at 0x11b380e50>}), (<A.4747966352>, {'task': <function <lambda> at 0x11b3823b0>})]
[]
[]
[(<A>, {'task': <function <lambda> at 0x11b3823b0>})]


### 绑定树

In [25]:
a.consumer_tree

{'provider': <A>,
 'consumer_tree': [{'consumer': <A C>,
   'binding_map': {'task': 'last_input'},
   'consumer_tree': {'provider': <A C>, 'consumer_tree': []}},
  {'consumer': <A.4747960448>,
   'binding_map': {'task': <function __main__.<lambda>(x)>},
   'consumer_tree': {'provider': <A.4747960448>, 'consumer_tree': []}},
  {'consumer': <A.4747966352>,
   'binding_map': {'task': <function __main__.<lambda>(x)>},
   'consumer_tree': {'provider': <A.4747966352>, 'consumer_tree': []}}]}

In [26]:
a.provider_tree

{'consumer': <A>, 'provider_tree': []}

In [27]:
c.provider_tree

{'consumer': <A.4747966352>,
 'provider_tree': [{'provider': <A>,
   'binding_map': {'task': <function __main__.<lambda>(x)>},
   'provider_tree': {'consumer': <A>, 'provider_tree': []}}]}

### 动态绑定
有时候，仅希望绑定关系短暂维持。例如，在调用函数中临时建立的绑定关系，希望每次重置。<br>
**这与实例声明时希望长期建立的绑定关系不同，称为动态绑定。**

**首先，重新声明一个常规绑定：**

In [28]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        self._last_input = prompt
        self._last_output = None

a = A(name="A")
c = A(name="C")

a.bind_consumers(c)
a._last_output = "A is great"
print("c.consumer_dict", c.consumer_dict)

c.consumer_dict {'last_output': 'A is great'}


**紧接着，我们申请一个 c 的动态绑定：**

In [29]:
c.bind_providers({"x": "I m x"}, dynamic=True)
c.consumer_dict

{'last_output': 'A is great', 'x': 'I m x'}

**我们申请 一个 c 的动态绑定：**<br>
此时，上一次执行的动态绑定并不会造成干扰。

In [30]:
c.bind_providers({"y": "I m y"}, dynamic=True)
c.consumer_dict

{'last_output': 'A is great', 'y': 'I m y'}

**如果绑定的是一个字典，会直接绑定字典的值**

In [31]:
c = A()
c.bind_providers({"task": "帮我写一首儿歌"})
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

c.consumer_dict {'task': '帮我写一首儿歌'}
c.provider_dict {'task': '帮我写一首儿歌'}


**如果绑定多个 Runnable 绑定相同变量，则以最后一个为准**

In [32]:
c = A()
c.bind_providers({"task": "帮我写一首儿歌"}, a)
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

c.consumer_dict {'task': '帮我写一首儿歌', 'last_output': 'A is great'}
c.provider_dict {'task': '帮我写一首儿歌', 'last_output': 'A is great'}


## 在容器实例内绑定

In [33]:
from illufly.types import Runnable

# A 有自己的计算函数，但绑定 B 之后可以将 B 的 provider_dict 作为入参实现动态 A 的动态计算
class A(Runnable):
    def call(self, prompt: str):
        self._last_input = prompt
        self._last_output = f'我听到B说： {self.consumer_dict["last_input"]} 来自A的{prompt}'

class B(Runnable):
    def __init__(self, obj: Runnable):
        super().__init__()
        # 将 A 绑定到自己
        obj.bind_providers(self)
        self.obj = obj

    def call(self, instruction):
        self._last_input = instruction
        self.obj.call("观察")
        print(self.obj.last_output)

a = A()
b = B(a)
b.call("今晚吃啥？")

我听到B说： 今晚吃啥？ 来自A的观察


In [34]:
print("a: ", end="")
print(a.consumer_dict, end=", ")
print(a.provider_dict)
print("b: ", end="")
print(b.consumer_dict, end=", ")
print(b.provider_dict)

a: {'last_input': '今晚吃啥？'}, {'last_input': '观察', 'last_output': '我听到B说： 今晚吃啥？ 来自A的观察'}
b: {}, {'last_input': '今晚吃啥？'}


In [3]:
a.providers

[(<B B.4501344448>, {})]